# Synthetic Time Series Data Generation

This notebook generates 10 different synthetic time series datasets for comparing traditional prediction intervals with conformal prediction methods.

Each dataset is designed to test different assumptions and weaknesses of traditional prediction interval methods.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima_process import arma_generate_sample
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('../data', exist_ok=True)

print("Setup complete!")

## Dataset 1: Baseline AR(1) with Gaussian Noise

**Purpose**: Ideal scenario where traditional methods should work well  
**Model**: y_t = 0.7 * y_{t-1} + ε, where ε ~ N(0, 1)  
**What's varying**: Nothing (baseline)

In [ ]:
# Dataset 1: Baseline AR(1) with Gaussian noise
np.random.seed(1)
n = 2500
ar_coef = 0.7

# Generate AR(1) process
y = np.zeros(n)
y[0] = np.random.normal(0, 1)
for t in range(1, n):
    y[t] = ar_coef * y[t-1] + np.random.normal(0, 1)

# Create DataFrame
df1 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'true_variance': 1.0  # constant variance
})

# Save to CSV
df1.to_csv('../data/dataset_1_baseline_ar1.csv', index=False)
print(f"Dataset 1 generated: {len(df1)} observations")
print(f"Mean: {df1['value'].mean():.3f}, Std: {df1['value'].std():.3f}")

In [ ]:
# Plot Dataset 1
plt.figure(figsize=(12, 4))
plt.plot(df1['timestamp'], df1['value'], linewidth=0.8)
plt.title('Dataset 1: Baseline AR(1) with Gaussian Noise')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Dataset 2: AR(1) with Heavy-Tailed Noise (Student-t)

**Purpose**: Test robustness to outliers and fat tails  
**Model**: y_t = 0.7 * y_{t-1} + ε, where ε ~ t(df=3)  
**What's varying**: Noise distribution (violates Gaussian assumption)

In [ ]:
# Dataset 2: AR(1) with heavy-tailed noise (Student-t)
np.random.seed(2)
n = 2500
ar_coef = 0.7
df_t = 3  # degrees of freedom for t-distribution

# Generate AR(1) process with t-distributed noise
y = np.zeros(n)
y[0] = stats.t.rvs(df_t)
for t in range(1, n):
    y[t] = ar_coef * y[t-1] + stats.t.rvs(df_t)

# Create DataFrame
df2 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'noise_type': 'student_t',
    'df': df_t
})

# Save to CSV
df2.to_csv('../data/dataset_2_heavy_tailed.csv', index=False)
print(f"Dataset 2 generated: {len(df2)} observations")
print(f"Mean: {df2['value'].mean():.3f}, Std: {df2['value'].std():.3f}")
print(f"Kurtosis: {stats.kurtosis(df2['value']):.3f} (higher than Gaussian)")

In [ ]:
# Plot Dataset 2
plt.figure(figsize=(12, 4))
plt.plot(df2['timestamp'], df2['value'], linewidth=0.8, color='orange')
plt.title('Dataset 2: AR(1) with Heavy-Tailed Noise (Student-t, df=3)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Dataset 3: AR(1) with Heteroscedastic Noise (GARCH-like)

**Purpose**: Test whether intervals adapt to changing volatility  
**Model**: y_t = 0.7 * y_{t-1} + σ_t * ε, where σ_t depends on recent squared residuals  
**What's varying**: Variance changes over time (volatility clustering)

In [ ]:
# Dataset 3: AR(1) with GARCH-like heteroscedastic noise
np.random.seed(3)
n = 2500
ar_coef = 0.7

# GARCH(1,1) parameters
alpha0 = 0.1
alpha1 = 0.15  # ARCH effect
beta1 = 0.75   # GARCH effect

# Generate AR(1) + GARCH process
y = np.zeros(n)
sigma2 = np.zeros(n)
sigma2[0] = 1.0
y[0] = np.random.normal(0, np.sqrt(sigma2[0]))

for t in range(1, n):
    # Update conditional variance (GARCH equation)
    sigma2[t] = alpha0 + alpha1 * (y[t-1] - ar_coef * y[t-2] if t > 1 else y[t-1])**2 + beta1 * sigma2[t-1]
    # Generate observation
    epsilon = np.random.normal(0, np.sqrt(sigma2[t]))
    y[t] = ar_coef * y[t-1] + epsilon

# Create DataFrame
df3 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'true_variance': sigma2
})

# Save to CSV
df3.to_csv('../data/dataset_3_garch.csv', index=False)
print(f"Dataset 3 generated: {len(df3)} observations")
print(f"Mean: {df3['value'].mean():.3f}, Std: {df3['value'].std():.3f}")
print(f"Variance range: [{df3['true_variance'].min():.3f}, {df3['true_variance'].max():.3f}]")

In [ ]:
# Plot Dataset 3
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax1.plot(df3['timestamp'], df3['value'], linewidth=0.8, color='green')
ax1.set_title('Dataset 3: AR(1) with GARCH-like Heteroscedastic Noise')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.plot(df3['timestamp'], df3['true_variance'], linewidth=0.8, color='red')
ax2.set_ylabel('Conditional Variance')
ax2.set_xlabel('Time')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 4: AR(1) with Time-Varying Variance (Deterministic)

**Purpose**: Test simpler heteroscedasticity with predictable pattern  
**Model**: y_t = 0.7 * y_{t-1} + σ_t * ε, where σ_t = 1 + 2*sin(2πt/500)  
**What's varying**: Variance follows a sinusoidal pattern

In [ ]:
# Dataset 4: AR(1) with time-varying variance (deterministic)
np.random.seed(4)
n = 2500
ar_coef = 0.7

# Time-varying standard deviation (sinusoidal pattern)
t = np.arange(n)
sigma_t = 1 + 2 * np.sin(2 * np.pi * t / 500)

# Generate AR(1) process with time-varying variance
y = np.zeros(n)
y[0] = np.random.normal(0, sigma_t[0])
for i in range(1, n):
    y[i] = ar_coef * y[i-1] + sigma_t[i] * np.random.normal(0, 1)

# Create DataFrame
df4 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'true_variance': sigma_t**2
})

# Save to CSV
df4.to_csv('../data/dataset_4_timevarying_variance.csv', index=False)
print(f"Dataset 4 generated: {len(df4)} observations")
print(f"Mean: {df4['value'].mean():.3f}, Std: {df4['value'].std():.3f}")
print(f"Variance range: [{df4['true_variance'].min():.3f}, {df4['true_variance'].max():.3f}]")

In [ ]:
# Plot Dataset 4
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax1.plot(df4['timestamp'], df4['value'], linewidth=0.8, color='purple')
ax1.set_title('Dataset 4: AR(1) with Deterministic Time-Varying Variance')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.plot(df4['timestamp'], df4['true_variance'], linewidth=0.8, color='red')
ax2.set_ylabel('Variance (sinusoidal)')
ax2.set_xlabel('Time')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 5: Regime-Switching Model

**Purpose**: Test adaptivity to distribution shifts  
**Model**: AR(1) with two regimes - low volatility (σ=0.5) and high volatility (σ=2)  
**What's varying**: Abrupt changes in volatility (concept drift)

In [ ]:
# Dataset 5: Regime-switching model
np.random.seed(5)
n = 2500
ar_coef = 0.7

# Define regimes
low_vol = 0.5
high_vol = 2.0

# Generate regime switches every 300-500 steps
regime = np.zeros(n)
regime_indicator = np.zeros(n)
current_pos = 0
current_regime = 0  # 0 = low vol, 1 = high vol

while current_pos < n:
    # Random duration between 300 and 500
    duration = np.random.randint(300, 501)
    end_pos = min(current_pos + duration, n)
    
    regime[current_pos:end_pos] = low_vol if current_regime == 0 else high_vol
    regime_indicator[current_pos:end_pos] = current_regime
    
    current_pos = end_pos
    current_regime = 1 - current_regime  # Switch regime

# Generate AR(1) process with regime-switching volatility
y = np.zeros(n)
y[0] = np.random.normal(0, regime[0])
for t in range(1, n):
    y[t] = ar_coef * y[t-1] + regime[t] * np.random.normal(0, 1)

# Create DataFrame
df5 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'regime': regime_indicator.astype(int),
    'true_variance': regime**2
})

# Save to CSV
df5.to_csv('../data/dataset_5_regime_switching.csv', index=False)
print(f"Dataset 5 generated: {len(df5)} observations")
print(f"Mean: {df5['value'].mean():.3f}, Std: {df5['value'].std():.3f}")
print(f"Regime changes: {np.sum(np.diff(df5['regime']) != 0)}")

In [ ]:
# Plot Dataset 5
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax1.plot(df5['timestamp'], df5['value'], linewidth=0.8, color='brown')
ax1.set_title('Dataset 5: Regime-Switching Model')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.fill_between(df5['timestamp'], 0, df5['regime'], alpha=0.5, color='gray', label='Regime')
ax2.set_ylabel('Regime (0=low, 1=high)')
ax2.set_xlabel('Time')
ax2.set_ylim([-0.1, 1.1])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 6: Non-Stationary with Trend Breaks

**Purpose**: Test how methods handle non-stationarity  
**Model**: AR(1) with 2-3 structural breaks where intercept changes  
**What's varying**: Mean shifts at random points

In [ ]:
# Dataset 6: Non-stationary with trend breaks
np.random.seed(6)
n = 2500
ar_coef = 0.7

# Define structural break points and intercepts
break_points = [0, 800, 1500, 2100, n]
intercepts = [0, 3, -2, 1]

# Generate AR(1) process with structural breaks
y = np.zeros(n)
break_indicator = np.zeros(n)
current_intercept = np.zeros(n)

for i in range(len(break_points) - 1):
    start = break_points[i]
    end = break_points[i + 1]
    current_intercept[start:end] = intercepts[i]
    break_indicator[start:end] = i

y[0] = intercepts[0] + np.random.normal(0, 1)
for t in range(1, n):
    # Find current intercept
    segment = int(break_indicator[t])
    intercept = intercepts[segment]
    y[t] = intercept + ar_coef * (y[t-1] - intercepts[segment-1] if t > 0 and segment > 0 else y[t-1]) + np.random.normal(0, 1)

# Create DataFrame
df6 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'intercept': current_intercept,
    'break_segment': break_indicator.astype(int)
})

# Save to CSV
df6.to_csv('../data/dataset_6_trend_breaks.csv', index=False)
print(f"Dataset 6 generated: {len(df6)} observations")
print(f"Mean: {df6['value'].mean():.3f}, Std: {df6['value'].std():.3f}")
print(f"Break points: {break_points[1:-1]}")

In [ ]:
# Plot Dataset 6
plt.figure(figsize=(12, 4))
plt.plot(df6['timestamp'], df6['value'], linewidth=0.8, color='navy')

# Mark break points
break_points_plot = [800, 1500, 2100]
for bp in break_points_plot:
    plt.axvline(x=bp, color='red', linestyle='--', alpha=0.5, linewidth=1)

plt.title('Dataset 6: Non-Stationary with Trend Breaks')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Dataset 7: AR(1) with Skewed Noise

**Purpose**: Test interval symmetry assumptions  
**Model**: y_t = 0.7 * y_{t-1} + ε, where ε ~ Skew-normal  
**What's varying**: Noise is asymmetric

In [ ]:
# Dataset 7: AR(1) with skewed noise
np.random.seed(7)
n = 2500
ar_coef = 0.7

# Skew-normal parameters
skewness = 5  # positive skewness

# Generate AR(1) process with skew-normal noise
y = np.zeros(n)
y[0] = stats.skewnorm.rvs(skewness)
for t in range(1, n):
    y[t] = ar_coef * y[t-1] + stats.skewnorm.rvs(skewness)

# Create DataFrame
df7 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'noise_type': 'skew_normal',
    'skewness_param': skewness
})

# Save to CSV
df7.to_csv('../data/dataset_7_skewed_noise.csv', index=False)
print(f"Dataset 7 generated: {len(df7)} observations")
print(f"Mean: {df7['value'].mean():.3f}, Std: {df7['value'].std():.3f}")
print(f"Skewness: {stats.skew(df7['value']):.3f}")

In [ ]:
# Plot Dataset 7
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(df7['timestamp'], df7['value'], linewidth=0.8, color='teal')
ax1.set_title('Dataset 7: AR(1) with Skewed Noise')
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.hist(df7['value'], bins=50, color='teal', alpha=0.7, edgecolor='black')
ax2.set_title('Distribution (showing skewness)')
ax2.set_xlabel('Value')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 8: Seasonal + Noise with Varying Seasonal Amplitude

**Purpose**: Test decomposition-based methods  
**Model**: y_t = trend + seasonal_t * amplitude_t + AR noise  
**What's varying**: Seasonality strength changes over time

In [ ]:
# Dataset 8: Seasonal + noise with varying seasonal amplitude
np.random.seed(8)
n = 2500
ar_coef = 0.5

# Time variable
t = np.arange(n)

# Trend component
trend = 0.01 * t

# Seasonal component (period = 50)
seasonal_period = 50
seasonal = np.sin(2 * np.pi * t / seasonal_period)

# Time-varying seasonal amplitude
amplitude_t = 1 + 2 * np.sin(2 * np.pi * t / 500)

# Generate AR(1) process for noise
noise = np.zeros(n)
noise[0] = np.random.normal(0, 0.5)
for i in range(1, n):
    noise[i] = ar_coef * noise[i-1] + np.random.normal(0, 0.5)

# Combine components
y = trend + seasonal * amplitude_t + noise

# Create DataFrame
df8 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'trend': trend,
    'seasonal_amplitude': amplitude_t,
    'seasonal_component': seasonal * amplitude_t
})

# Save to CSV
df8.to_csv('../data/dataset_8_varying_seasonal.csv', index=False)
print(f"Dataset 8 generated: {len(df8)} observations")
print(f"Mean: {df8['value'].mean():.3f}, Std: {df8['value'].std():.3f}")
print(f"Seasonal amplitude range: [{df8['seasonal_amplitude'].min():.3f}, {df8['seasonal_amplitude'].max():.3f}]")

In [ ]:
# Plot Dataset 8
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax1.plot(df8['timestamp'], df8['value'], linewidth=0.8, color='magenta')
ax1.set_title('Dataset 8: Seasonal with Varying Amplitude')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.plot(df8['timestamp'], df8['seasonal_amplitude'], linewidth=0.8, color='red')
ax2.set_ylabel('Seasonal Amplitude')
ax2.set_xlabel('Time')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 9: Mixture of Gaussians Noise

**Purpose**: Test robustness to contamination  
**Model**: y_t = 0.7 * y_{t-1} + ε, where ε ~ 0.8*N(0,1) + 0.2*N(0,5)  
**What's varying**: Bimodal distribution with occasional large shocks

In [ ]:
# Dataset 9: Mixture of Gaussians noise
np.random.seed(9)
n = 2500
ar_coef = 0.7

# Mixture parameters
p_normal = 0.8  # probability of normal component
p_shock = 0.2   # probability of shock component

# Generate mixture noise
def mixture_gaussian():
    if np.random.rand() < p_normal:
        return np.random.normal(0, 1)
    else:
        return np.random.normal(0, 5)

# Generate AR(1) process with mixture noise
y = np.zeros(n)
mixture_indicator = np.zeros(n)
y[0] = mixture_gaussian()
for t in range(1, n):
    epsilon = mixture_gaussian()
    mixture_indicator[t] = 1 if abs(epsilon) > 2 else 0  # indicator for large shocks
    y[t] = ar_coef * y[t-1] + epsilon

# Create DataFrame
df9 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'shock_indicator': mixture_indicator.astype(int)
})

# Save to CSV
df9.to_csv('../data/dataset_9_mixture_gaussian.csv', index=False)
print(f"Dataset 9 generated: {len(df9)} observations")
print(f"Mean: {df9['value'].mean():.3f}, Std: {df9['value'].std():.3f}")
print(f"Large shocks: {df9['shock_indicator'].sum()} ({100*df9['shock_indicator'].mean():.1f}%)")

In [ ]:
# Plot Dataset 9
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(df9['timestamp'], df9['value'], linewidth=0.8, color='darkred')
shock_times = df9[df9['shock_indicator'] == 1]['timestamp']
shock_values = df9[df9['shock_indicator'] == 1]['value']
ax1.scatter(shock_times, shock_values, color='red', s=20, alpha=0.5, label='Large shocks')
ax1.set_title('Dataset 9: AR(1) with Mixture of Gaussians Noise')
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.hist(df9['value'], bins=50, color='darkred', alpha=0.7, edgecolor='black')
ax2.set_title('Distribution (bimodal/contaminated)')
ax2.set_xlabel('Value')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dataset 10: Long-Memory Process (ARFIMA)

**Purpose**: Test methods on slowly decaying correlations  
**Model**: Fractionally integrated process with d ≈ 0.3  
**What's varying**: Long-range dependence (autocorrelation structure)

In [ ]:
# Dataset 10: Long-memory process (ARFIMA)
np.random.seed(10)
n = 2500
d = 0.3  # fractional differencing parameter

# Generate fractional differencing coefficients
def fracdiff_coef(d, n_coefs):
    """Generate fractional differencing coefficients"""
    coefs = np.zeros(n_coefs)
    coefs[0] = 1
    for k in range(1, n_coefs):
        coefs[k] = coefs[k-1] * (k - 1 - d) / k
    return coefs

# Get coefficients
max_lag = 100
coefs = fracdiff_coef(d, max_lag)

# Generate ARFIMA process
# Start with white noise
white_noise = np.random.normal(0, 1, n + max_lag)

# Apply fractional integration (inverse of fractional differencing)
y = np.zeros(n)
for t in range(n):
    # Sum over past white noise with fractional weights
    y[t] = sum(coefs[k] * white_noise[t + max_lag - k] for k in range(min(t + 1, max_lag)))

# Create DataFrame
df10 = pd.DataFrame({
    'timestamp': range(n),
    'value': y,
    'fracdiff_param': d
})

# Save to CSV
df10.to_csv('../data/dataset_10_arfima.csv', index=False)
print(f"Dataset 10 generated: {len(df10)} observations")
print(f"Mean: {df10['value'].mean():.3f}, Std: {df10['value'].std():.3f}")
print(f"Fractional differencing parameter d: {d}")

In [ ]:
# Plot Dataset 10
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(df10['timestamp'], df10['value'], linewidth=0.8, color='darkgreen')
ax1.set_title('Dataset 10: ARFIMA Process (Long Memory)')
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

# Compute and plot autocorrelation
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df10['value'], ax=ax2, color='darkgreen')
ax2.set_title('Autocorrelation (slowly decaying)')
ax2.set_xlabel('Lag')
ax2.set_xlim([0, 100])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary Table

Overview of all generated datasets and what assumptions they test:

In [ ]:
# Create summary table
summary_data = {
    'Dataset': [
        '1. Baseline AR(1)',
        '2. Heavy-tailed noise',
        '3. GARCH-like',
        '4. Time-varying variance',
        '5. Regime-switching',
        '6. Trend breaks',
        '7. Skewed noise',
        '8. Varying seasonal',
        '9. Mixture of Gaussians',
        '10. ARFIMA'
    ],
    'What\'s Varying': [
        'Nothing (baseline)',
        'Noise distribution (heavy tails)',
        'Variance (GARCH)',
        'Variance (deterministic pattern)',
        'Regime switches',
        'Trend breaks',
        'Skewed noise',
        'Seasonal amplitude',
        'Bimodal noise',
        'Long memory'
    ],
    'Traditional PI Weakness Tested': [
        'None — should work well',
        'Gaussian assumption',
        'Constant variance assumption',
        'Constant variance assumption',
        'Stationarity assumption',
        'Stationarity assumption',
        'Symmetric distribution assumption',
        'Fixed seasonal pattern assumption',
        'Unimodal distribution assumption',
        'Short-range dependence assumption'
    ],
    'File': [
        'dataset_1_baseline_ar1.csv',
        'dataset_2_heavy_tailed.csv',
        'dataset_3_garch.csv',
        'dataset_4_timevarying_variance.csv',
        'dataset_5_regime_switching.csv',
        'dataset_6_trend_breaks.csv',
        'dataset_7_skewed_noise.csv',
        'dataset_8_varying_seasonal.csv',
        'dataset_9_mixture_gaussian.csv',
        'dataset_10_arfima.csv'
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("SUMMARY OF SYNTHETIC DATASETS")
print("="*100)
print(summary_df.to_string(index=False))
print("="*100)
print(f"\nAll datasets saved to ../data/ directory")
print(f"Total datasets: {len(summary_df)}")
print(f"Total observations per dataset: ~2500")

## Comprehensive Visualization

Plot all 10 datasets in a single grid for comparison:

In [ ]:
# Comprehensive visualization of all 10 datasets
fig, axes = plt.subplots(5, 2, figsize=(15, 18))
axes = axes.flatten()

datasets = [
    (df1, '1. Baseline AR(1)', 'blue'),
    (df2, '2. Heavy-tailed (Student-t)', 'orange'),
    (df3, '3. GARCH-like', 'green'),
    (df4, '4. Time-varying Variance', 'purple'),
    (df5, '5. Regime-switching', 'brown'),
    (df6, '6. Trend Breaks', 'navy'),
    (df7, '7. Skewed Noise', 'teal'),
    (df8, '8. Varying Seasonal', 'magenta'),
    (df9, '9. Mixture of Gaussians', 'darkred'),
    (df10, '10. ARFIMA (Long Memory)', 'darkgreen')
]

for idx, (df, title, color) in enumerate(datasets):
    axes[idx].plot(df['timestamp'], df['value'], linewidth=0.6, color=color)
    axes[idx].set_title(title, fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Time', fontsize=8)
    axes[idx].set_ylabel('Value', fontsize=8)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(labelsize=8)

plt.suptitle('Synthetic Time Series Datasets for Conformal Prediction Comparison', 
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../data/all_datasets_overview.png', dpi=150, bbox_inches='tight')
print("Comprehensive plot saved to ../data/all_datasets_overview.png")
plt.show()